# <center>4. Sparsification of Dose Influence Matrix </center>

### PortPy provides full dense influence matrix (i.e., including all scattering components) and a truncated sparse version (used for computational efficiency). This example clarifies the relationship and differences between these matrices by showing the followings:

1. Generating a plan using the sparse matrix (default matrix in PortPy)
2. Calculating the full dose for the plan using the full matrix and visualize the discrepancy between sparse and full dose.
3. Manually calculating the sparse matrix from the full matrix

### 1) Generating a plan using the sparse matrix (default matrix in PortPy)

In [1]:
import sys
sys.path.append('..')

In [2]:
import portpy.photon as pp
import matplotlib.pyplot as plt
import numpy as np

### 2) Calculating the full dose for the plan using the full matrix and visualize the discrepancy between sparse and full dose.

In [ ]:
# Note: It is often computationally impractical to use the full matrix for optimization. We just use the
    #   full matrix to calculate the dose for the solution obtained by sparse matrix and show the resultant discrepancy

# create plan_full object by specifying load_inf_matrix_full=True
beams_full = pp.Beams(data, load_inf_matrix_full=True)
# load influence matrix based upon beams and structure set
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
# use the full influence matrix to calculate the dose for the plan obtained by sparse matrix
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

# Visualize the DVH discrepancy
struct_names = ['PTV', 'ESOPHAGUS', 'HEART', 'CORD']
fig, ax = plt.subplots(figsize=(12, 8))
# Turn on norm flag for same normalization for sparse and full dose.
ax = pp.Visualization.plot_dvh(plan_sparse, dose_1d=dose_sparse_1d, struct_names=struct_names, style='solid', ax=ax, norm_flag=True)
ax = pp.Visualization.plot_dvh(plan_full, dose_1d=dose_full_1d, struct_names=struct_names, style='dotted', ax=ax, norm_flag=True)
ax.set_title('- Sparse    .. Full')
plt.show()
print('Done')


### 3) Manually calculating the sparse matrix from the full matrix

In [ ]:
# The sparse and full matrices are both pre-calculated and included in PorPy data.
#   The sparse matrix; however, was obtained by simply zeroing out the small elements in the full matrix that were
#   less than a threshold specified in "my_plan.inf_matrix.sparse_tol". Here, we manually generate the sparse
#   matrix from the full matrix using this threshold to clarify the process

#  Get A_sparse and A_full
A_full = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A

# Get the threshold value used by PortPy to truncate the matrix
# sparse tol is 1% of the maximum of influence matrix of planner beams
sparse_tol = plan_sparse.inf_matrix.sparse_tol
# sparse_tol = 0.01*np.amax(A_full)

# Truncate the full matrix
A_full[A_full <= sparse_tol] = 0
test = np.abs(A_full - A_sparse.todense()) <= 1e-3

# Check if both influence matrices agree
assert test.all()